In [ ]:
import os
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir(os.pardir)

import matplotlib.pyplot as plt
import numpy as np

from src.utils.dataloader import load_ear_data
from src.utils.viz import set_axes_equal

In [ ]:
def export_pc(df):
    return np.c_[df['x [mm]'].to_numpy(),
                 df['y [mm]'].to_numpy(),
                 df['z [mm]'].to_numpy()]


def scale_pc(xyz, scaler):
    start_point = np.min(xyz, axis=0)
    d_xyz = np.diff(xyz, axis=0)
    return np.cumsum(np.vstack((start_point, d_xyz * scaler)), axis=0)


def export_fields(df):
    Ex = df['ExRe [V/m]'].to_numpy() + 1j * df['ExIm [V/m]'].to_numpy()
    Ey = df['EyRe [V/m]'].to_numpy() + 1j * df['EyIm [V/m]'].to_numpy()
    Ez = df['EzRe [V/m]'].to_numpy() + 1j * df['EzIm [V/m]'].to_numpy()
    Hx = df['HxRe [A/m]'].to_numpy() + 1j * df['HxIm [A/m]'].to_numpy()
    Hy = df['HyRe [A/m]'].to_numpy() + 1j * df['HyIm [A/m]'].to_numpy()
    Hz = df['HzRe [A/m]'].to_numpy() + 1j * df['HzIm [A/m]'].to_numpy()
    return ((Ex, Ey, Ez), (Hx, Hy, Hz))


def poynting_vector(E, H):
    return (E[1] * H[2].conjugate() - E[2] * H[1].conjugate(),
            E[2] * H[0].conjugate() - E[0] * H[2].conjugate(),
            E[0] * H[1].conjugate() - E[1] * H[0].conjugate())


def plot_2d(xy_dict, figsize=plt.rcParams['figure.figsize']):
    fig = plt.figure(figsize=figsize)
    ax = plt.axes()
    keys = list(xy_dict.keys())
    values = list(xy_dict.values())
    cs = ax.scatter(values[0], values[1], c=values[2])
    cbar = fig.colorbar(cs)
    ax.set(xlabel=keys[0], ylabel=keys[1])
    cbar.ax.set_ylabel(keys[2])
    ax.axis('equal')
    fig.tight_layout()
    return fig


def plot_3d(xyz_dict, figsize=(7, 7), elev=20, azim=45, c=None):
    fig = plt.figure(figsize=figsize)
    ax = plt.axes(projection ='3d')
    keys = list(xyz_dict.keys())
    values = list(xyz_dict.values())
    if (len(values) == 4) and not(c):
        cs = ax.scatter(values[0], values[1], values[2], c=values[3])
        cbar = fig.colorbar(cs, shrink=0.5)
        cbar.ax.set_ylabel(keys[3])
    else:
        if not(c):
            c = 'k'
        cs = ax.scatter(values[0], values[1], values[2], c=c)
    ax.set(xlabel=keys[0], ylabel=keys[1], zlabel=keys[2])
    ax = set_axes_equal(ax)
    ax.view_init(elev, azim)
    fig.tight_layout()
    return fig

In [ ]:
df = load_ear_data('te', 26, 1e-3)
df

In [ ]:
xyz = export_pc(df)
xyz.shape

In [ ]:
E, H = export_fields(df)
len(E), len(H)

In [ ]:
Sx, Sy, Sz = poynting_vector(E, H)
S_dist_cpx = np.sqrt(Sx ** 2 + Sy ** 2 + Sz ** 2)
S_dist_abs = np.abs(S_dist_cpx)

In [ ]:
fig = plot_3d({'x [mm]': xyz[:, 0],
               'y [mm]': xyz[:, 1],
               'z [mm]': xyz[:, 2],
               'S [W/m2]': S_dist_abs})

In [ ]:
fig = plot_2d({'y [mm]': xyz[:, 1],
               'z [mm]': xyz[:, 2],
               'S [W/m2]': S_dist_abs})

In [ ]:
current_ear_length = xyz[:, 1].max() - xyz[:, 1].min()
target_ear_length = lambda age: 59.17 + 0.239 * age  # JPRAS (2011) 64, 41--47
scaler = target_ear_length(age=35) / current_ear_length
xyz_scaled = scale_pc(xyz, scaler)
fig = plot_3d({'x [mm]': xyz_scaled[:, 0],
               'y [mm]': xyz_scaled[:, 1],
               'z [mm]': xyz_scaled[:, 2],
               'S [W/m2]': S_dist_abs})